NAMA : RACHMAWATI DWINANTI PUTRI
NIM : 09011281823064
KELAS : SK5B

Notebook ini pada dasarnya mencakup tutorial dasar untuk Support Vector Machine. Saya akan menggunakan data prediksi seluler untuk excerise ini. 

Catatan: 
    1) Kumpulan data ini bukan kumpulan data yang bagus untuk mempraktikkan klasifikasi SVM, saya menggunakannya untuk mencoba SVM dengan sederhana. 
    2) Jika Anda memiliki kumpulan data yang lebih baik maka saya akan merekomendasikan menggunakan itu atau kumpulan Data IRIS sangat bagus untuk masalah ini. 

Topik di bawah ini dibahas dalam Kernal ini. 
- Prepocessing data 
- Analisis nilai target
- SVM
- SVM Linear
- Regresor SV
- Non Linear SVM dengan kernal - RBF (catatan: Anda juga dapat mencoba poli)
- SVR Non Linear 

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**DATA PREPROCESSING**

In [3]:
df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
df.head()

In [4]:
# checking if there is any missing value
df.isnull().sum().max()
df.columns

**TARGET VALUE ANALYSIS**

In [5]:
#understanding the predicted value - which is hot encoded, in real life price won't be hot encoded.
df['price_range'].describe(), df['price_range'].unique()

# there are 4 classes in the predicted value

In [6]:
corrmat = df.corr()
f,ax = plt.subplots(figsize=(12,10))
sns.heatmap(corrmat,vmax=0.8,square=True,annot=True,annot_kws={'size':8})

In [ ]:
f, ax = plt.subplots(figsize=(10,4))
plt.scatter(y=df['price_range'],x=df['battery_power'],color='red')
plt.scatter(y=df['price_range'],x=df['ram'],color='Green')
plt.scatter(y=df['price_range'],x=df['n_cores'],color='blue')
plt.scatter(y=df['price_range'],x=df['mobile_wt'],color='orange')
# clearly we can see that each of the category has different set of value ranges 

In [ ]:
# Try plots using seaborn
#sns.swarmplot(x='battery_power',y='ram',data=df,hue='price_range')
#plt.show()

In [ ]:
#sns.pairplot(df,size=2.5)
#plt.show()

Sekarang dalam kumpulan data tidak perlu membuat variabel dummy atau menangani data yang hilang karena kumpulan data tidak memiliki data yang hilang

**SUPPORT VECTOR MACHINES AND METHODS : **

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

y_t = np.array(df['price_range'])
X_t = df
X_t = df.drop(['price_range'],axis=1)
X_t = np.array(X_t)

print("shape of Y :"+str(y_t.shape))
print("shape of X :"+str(X_t.shape))

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X_t)

In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))

In [9]:
for this_C in [1,3,5,10,40,60,80,100]:
    clf = SVC(kernel='linear',C=this_C).fit(X_train,Y_train)
    scoretrain = clf.score(X_train,Y_train)
    scoretest  = clf.score(X_test,Y_test)
    print("Linear SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))

In [10]:
from sklearn.model_selection import cross_val_score,StratifiedKFold,LeaveOneOut
clf1 = SVC(kernel='linear',C=20).fit(X_train,Y_train)
scores = cross_val_score(clf1,X_train,Y_train,cv=5)
strat_scores = cross_val_score(clf1,X_train,Y_train,cv=StratifiedKFold(5,random_state=10,shuffle=True))
#Loo = LeaveOneOut()
#Loo_scores = cross_val_score(clf1,X_train,Y_train,cv=Loo)
print("The Cross Validation Score :"+str(scores))
print("The Average Cross Validation Score :"+str(scores.mean()))
print("The Stratified Cross Validation Score :"+str(strat_scores))
print("The Average Stratified Cross Validation Score :"+str(strat_scores.mean()))
#print("The LeaveOneOut Cross Validation Score :"+str(Loo_scores))
#print("The Average LeaveOneOut Cross Validation Score :"+str(Loo_scores.mean()))

In [24]:
from sklearn.dummy import DummyClassifier

for strat in ['stratified', 'most_frequent', 'prior', 'uniform']:
    dummy_maj = DummyClassifier(strategy=strat).fit(X_train,Y_train)
    print("Train Stratergy :{} \n Score :{:.2f}".format(strat,dummy_maj.score(X_train,Y_train)))
    print("Test Stratergy :{} \n Score :{:.2f}".format(strat,dummy_maj.score(X_test,Y_test)))

In [25]:
# plotting the decision boundries for the data 
#converting the data to array for plotting. 
X = np.array(df.iloc[:,[0,13]])
y = np.array(df['price_range'])
print("Shape of X:"+str(X.shape))
print("Shape of y:"+str(y.shape))
X = scaler.fit_transform(X)

In [26]:
# custome color maps
cm_dark = ListedColormap(['#ff6060', '#8282ff','#ffaa00','#fff244','#4df9b9','#76e8fc','#3ad628'])
cm_bright = ListedColormap(['#ffafaf', '#c6c6ff','#ffaa00','#ffe2a8','#bfffe7','#c9f7ff','#9eff93'])

In [28]:
plt.scatter(X[:,0],X[:,1],c=y,cmap=cm_dark,s=10,label=y)
plt.show()

In [27]:
h = .02  # step size in the mesh
C_param = 1 # No of neighbours
for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf1 = SVC(kernel='linear',C=C_param)
    clf1.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min()-.20, X[:, 0].max()+.20
    y_min, y_max = X[:, 1].min()-.20, X[:, 1].max()+.20
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf1.predict(np.c_[xx.ravel(), yy.ravel()])   # ravel to flatten the into 1D and c_ to concatenate 

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cm_bright)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_dark,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("SVM Linear Classification (kernal = linear, Gamma = '%s')"% (C_param))

plt.show()

In [29]:
print("The score of the above :"+str(clf1.score(X,y)))

In [30]:
# Linear Support vector machine with only C Parameter 
from sklearn.svm import LinearSVC

for this_C in [1,3,5,10,40,60,80,100]:
    clf2 = LinearSVC(C=this_C).fit(X_train,Y_train)
    scoretrain = clf2.score(X_train,Y_train)
    scoretest  = clf2.score(X_test,Y_test)
    print("Linear SVM value of C:{}, training score :{:2f} , Test Score: {:2f} \n".format(this_C,scoretrain,scoretest))



Rupanya kami mendapat skor yang lebih baik dengan SVC di mana kami mendefinisikan kernal sebagai linier daripada hanya dengan LinearSVC

Kelas LinearSVC didasarkan pada pustaka liblinear, yang mengimplementasikan algoritme yang dioptimalkan untuk SVM linier.
1. Ini tidak mendukung trik kernel, tetapi berskala hampir linier dengan jumlah contoh pelatihan dan jumlah fitur: kompleksitas waktu pelatihannya kira-kira O (m × n).

Kelas SVC didasarkan pada pustaka libsvm, yang mengimplementasikan algoritme yang mendukung trik kernel.
1. Kompleksitas waktu pelatihan biasanya antara O (m2 × n) dan O (m3 × n).
1. LinearSVC jauh lebih cepat daripada SVC (kernel = "linear")

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='linear',C=1,epsilon=.01).fit(X_train,Y_train)
print("{:.2f} is the accuracy of the SV Regressor".format(svr.score(X_train,Y_train)))

* SVM mendukung regresi linier dan nonlinier.
* Regresi SVM mencoba memasukkan sebanyak mungkin contoh pada batas keputusan sambil membatasi pelanggaran margin.
* Lebar batas keputusan dikontrol oleh hyperparameter ϵ.

**NON LINEAR SVM **

Metode untuk Menangani hubungan Non linier dalam kumpulan data kami adalah dengan menggunakan polinomial Kernal atau menggunakan fungsi kesamaan dengan SVM kami.

Kami akan menggunakan fungsi Gaussian Radial Basis Function (RBF) untuk hal yang sama. untuk menangani ini di Sklearn ada hyperparameter Gamma.
Periksa Fungsi RBF Gausian - untuk info lebih lanjut.

Secara teknis, parameter gamma merupakan kebalikan dari deviasi standar kernel RBF (fungsi Gaussian), yang digunakan sebagai ukuran kemiripan antara dua titik. Secara intuitif, nilai gamma yang kecil menentukan fungsi Gaussian dengan varian yang besar. Dalam hal ini, dua titik dapat dianggap serupa meskipun jauh dari satu sama lain. **In the other hand, a large gamma value means define a Gaussian function with a small variance and in this case, two points are considered similar just if they are close to each other.**

Initution:  kami membuat landmark yang berbeda dan kemudian memeriksa seberapa jauh contoh pelatihan dari landmark. Dalam prakteknya, jika kita memiliki n contoh pelatihan maka kita akan memiliki n landmark dan kita akan membuat satu set fitur nilai n dengan n landmark. Ketika contoh pelatihan paling dekat dengan tengara, nilai variansnya akan kecil dan bila jauh nilainya akan besar dan karenanya kita akan mengasosiasikan contoh terdekat dengan tengara dengan 1 dan yang jauh dengan 0. Kemampuan ini membuat SVM sangat kuat.

In [31]:
# SMV with RBF KERNAL AND ONLY C PARAMETER 

for this_C in [1,5,10,25,50,100]:
    clf3 = SVC(kernel='rbf',C=this_C).fit(X_train,Y_train)
    clf3train = clf3.score(X_train,Y_train)
    clf3test  = clf3.score(X_test,Y_test)
    print("SVM for Non Linear \n C:{} Training Score : {:2f} Test Score : {:2f}\n".format(this_C,clf3train,clf3test))

In [32]:
h = .02  # step size in the mesh
C_param = 1 # No of neighbours
for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf1 = SVC(kernel='rbf',C=C_param)
    clf1.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf1.predict(np.c_[xx.ravel(), yy.ravel()])   # ravel to flatten the into 1D and c_ to concatenate 

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cm_bright)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_dark,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("SVM Linear Classification (kernal = linear, Gamma = '%s')"% (C_param))

plt.show()

In [33]:
# SVM WITH RBF KERNAL, C AND GAMMA HYPERPARAMTER 
for this_gamma in [.1,.5,.10,.25,.50,1]:
    for this_C in [1,5,7,10,15,25,50]:
        clf3 = SVC(kernel='rbf',C=this_C,gamma=this_gamma).fit(X_train,Y_train)
        clf3train = clf3.score(X_train,Y_train)
        clf3test  = clf3.score(X_test,Y_test)
        print("SVM for Non Linear \n Gamma: {} C:{} Training Score : {:2f} Test Score : {:2f}\n".format(this_gamma,this_C,clf3train,clf3test))

In [ ]:
# grid search method 
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1,5,7,10,15,25,50],
              'gamma': [.1,.5,.10,.25,.50,1]}
GS = GridSearchCV(SVC(kernel='rbf'),param_grid,cv=5)

In [ ]:
GS.fit(X_train,Y_train)

In [ ]:
print("the parameters {} are the best.".format(GS.best_params_))
print("the best score is {:.2f}.".format(GS.best_score_))

In [ ]:
# Kernalized SVM machine 
svr2 = SVR(degree=2,C=100,epsilon=.01).fit(X_train,Y_train)
print("{:.2f} is the accuracy of the SV Regressor".format(svr2.score(X_train,Y_train)))

Kita dapat melihat bahwa regressor Support Vector Machine kernalisasi memberikan akurasi yang lebih baik daripada SVM Regresor Linear (non kernal) sebelumnya. Bagaimanapun juga, perlu memahami data yang sedang dikerjakan sebelum mencoba berbagai metode. Teknik validasi silang berguna.

Saya selanjutnya dapat menambahkan teknik Validasi Silang untuk Anda gunakan.

In [ ]:
test = test.drop(['id'],axis=1)
test.head()

In [ ]:
test_mat = np.array(test)
test = scaler.fit_transform(test_mat)

In [ ]:
clf4 = SVC(kernel='rbf',C=25,gamma=.1).fit(X_train,Y_train)
prediction = clf4.predict(test_mat)
pred = pd.DataFrame(prediction)
pred.head()

In [ ]:
prediction = svr2.predict(test_mat)
pred = pd.DataFrame(prediction)
pred.head()

Kami telah memperkirakan nilai set pengujian yang diberikan kepada kami dalam kumpulan data dan kami dapat dari 2 blok sebelumnya bahwa prediksi kami cukup akurat.